In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

from ib_insync import util
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>
from src.utils import Vars, get_pickle

_vars = Vars(MARKET)
PORT = port = _vars.PORT

DATAPATH = ROOT / 'data' / MARKET

In [4]:
df = get_pickle(DATAPATH / 'df_all_opts.pkl')

In [6]:
df[~df.expPrice.isnull() & ~df.close.isnull()]

,conId,symbol,expiry,strike,right,undId,dte,undPrice,iv,sigma,...,askSize,lastSize,volume,high,low,bid,ask,last,close,expPrice
88,672851539,EMR,20240126,107.0,C,6842,29.025608,97.775,0.166671,4.595505,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.04,241.15
90,674171867,ENPH,20240202,185.0,C,105368327,36.025617,136.450,0.566232,24.273217,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.94,160.90
91,674183164,EOG,20240419,160.0,C,6890,113.025618,124.515,0.255136,17.678083,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.40,531.45
92,637741023,EQT,20240119,46.0,C,57698865,22.025611,39.215,0.342952,3.303720,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.09,52.55
93,670226514,EQT,20240126,47.0,C,57698865,29.025611,39.215,0.342952,3.792538,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.09,59.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,666656534,WM,20240719,145.0,P,13379,204.024963,181.425,0.131171,17.792247,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,1.23,1642.15
506,611039594,WFC,20240621,32.5,P,10375,176.024976,49.370,0.243528,8.349360,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.22,135.55
507,657953707,WFC,20250620,20.0,P,10375,540.024976,49.370,0.243528,14.624229,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.15,35.95
508,674040903,WMB,20240202,31.0,P,13805,36.024963,35.120,0.185298,2.044473,...,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,0.04,80.30
